In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import storage
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import gcsfs
import json
import firecloud.api
from glob import glob
import dalmatian as dm
import pandas as pd

In [ ]:
align_res = []
for align in glob("*alignment*"):
    align_res.append(pd.read_table(align, skiprows=6).loc[:, ['PF_HQ_ALIGNED_READS']])
align_res = pd.concat(align_res, axis=1)   
align_res.columns = list(map(lambda x:x.split('.')[0], glob("*alignment*")))

In [ ]:
#wm = dm.WorkspaceManager("broad-genomics-data/DepMap_WGS")
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")
allstat = wm.get_samples()

In [ ]:
for col in allstat.columns:
    if 'bam' in col:
        print(col)

In [ ]:
allstat_use = allstat.loc[:, ['internal_bam_filepath', 'internal_bam_filepath_alt','source', 'sex', 'primary_disease', 'tumor_bam_alignment_summary_metrics']]

In [ ]:
allstat_use.loc[allstat_use.loc[:,'tumor_bam_alignment_summary_metrics'].isnull(), :]['source'].value_counts()

In [ ]:
for alt in allstat_use['internal_bam_filepath'].dropna():
    print(alt)
    break

In [ ]:
allstat_use.loc[allstat_use.loc[:,'tumor_bam_alignment_summary_metrics'].isnull(), :].to_csv("CCLF.bam_list")

In [ ]:
cclf_bams = allstat_use.loc[allstat_use.loc[:,'tumor_bam_alignment_summary_metrics'].isnull(), ['internal_bam_filepath_alt', 'internal_bam_filepath']]

In [ ]:
cclf_bams.head()

In [ ]:
cclf_bams = pd.DataFrame(cclf_bams)
cclf_bams.loc[:, 'bai'] = cclf_bams.internal_bam_filepath_alt.str.replace(".bam", ".bai", regex=False)

In [ ]:
cclf_bams.loc[:, 'no_alt_bai'] = cclf_bams.internal_bam_filepath.str.replace(".bam", ".bai", regex=False)

In [ ]:
cclf_bams.loc[["CDS-VNR5t7", "CDS-CYVPOD", "CDS-aSwDFc"]]

In [ ]:
cclf_bams.shape

In [ ]:
bait = {"AGILENT": "gs://ccleparams/references/intervals/agilent_hg38_lifted_chrXY.no_header.list",
        "ICE": "gs://ccleparams/references/intervals/ice_hg38_lifted_chrXY.no_header.list"}

In [ ]:
bait

In [ ]:
bait_info = pd.read_table("baits_info_clean.tsv", header=None)

In [ ]:
bait_info.head()

In [ ]:
bait_info.shape

In [ ]:
import numpy as np
baits_id = [dict(zip(bait_info.iloc[:, 0], bait_info.iloc[:, 5])).get(cds_id, 'NA') for cds_id in cclf_bams.index]

In [ ]:
baits_id[:5]

In [ ]:
baits = [bait.get(b, 'NA') for b in baits_id]

In [ ]:
baits[:5]

In [ ]:
beds = [bait.get(b, 'NA').replace('.list', '.bed') for b in baits_id]

In [ ]:
beds[:5]

In [ ]:
cclf_bams.head()

In [ ]:
cclf_bams.shape, len(baits), len(beds)

# Test three alt-aware bam for QC steps

In [ ]:
jsons = {'BamMetrics.bams': list(cclf_bams.loc[:, 'internal_bam_filepath_alt'].values[:3]),
         'BamMetrics.bamIndexes': list(cclf_bams.loc[:, 'bai'].values[:3]),
         'BamMetrics.bait_interval': baits[:3], 
         'BamMetrics.targetBed': beds[:3],
         'BamMetrics.referenceFasta': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta',
         'BamMetrics.referenceFastaDict': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dict',
         'BamMetrics.referenceFastaFai': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta.fai'}

In [ ]:
import json
with open("input_cclf.json", "w") as js:
    json.dump(jsons, js)

In [ ]:
cclf_bams.shape

In [ ]:
cclf_bams.loc[:, 'bait'] = baits
cclf_bams.loc[:, 'bed'] = beds

In [ ]:
cclf_bams.head()

In [ ]:
cclf_bams.loc[cclf_bams.bait=='NA', :]

In [ ]:
cclf_bams = cclf_bams.dropna()

In [ ]:
cclf_bams.tail()

In [ ]:
cclf_bams

# Double check no-alt bams

In [ ]:
jsons = {'BamMetrics.bams': list(cclf_bams.loc[:, 'internal_bam_filepath'].values),
         'BamMetrics.bamIndexes': list(cclf_bams.loc[:, 'no_alt_bai'].values),
         'BamMetrics.bait_interval': list(cclf_bams.loc[:, 'bait'].values), 
         'BamMetrics.targetBed': list(cclf_bams.loc[:, 'bed'].values),
         'BamMetrics.referenceFasta': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta',
         'BamMetrics.referenceFastaDict': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dict',
         'BamMetrics.referenceFastaFai': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta.fai'}

In [ ]:
import json
with open("input_cclf_all_noalt.json", "w") as js:
    json.dump(jsons, js)

# Validate on all alt-aware bams

In [ ]:
jsons = {'BamMetrics.bams': list(cclf_bams.loc[:, 'internal_bam_filepath_alt'].values),
         'BamMetrics.bamIndexes': list(cclf_bams.loc[:, 'bai'].values),
         'BamMetrics.bait_interval': list(cclf_bams.loc[:, 'bait'].values), 
         'BamMetrics.targetBed': list(cclf_bams.loc[:, 'bed'].values),
         'BamMetrics.referenceFasta': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta',
         'BamMetrics.referenceFastaDict': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dict',
         'BamMetrics.referenceFastaFai': 'gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta.fai'}

In [ ]:
len(jsons['BamMetrics.bams'])

In [ ]:
import json
with open("input_cclf_all.json", "w") as js:
    json.dump(jsons, js)

In [ ]:
!pwd && ls input_cclf_all.json

In [ ]:
allstat_use.loc[~allstat_use.loc[:,'tumor_bam_alignment_summary_metrics'].isnull(), :]['source'].value_counts()

In [ ]:
allstat_use.loc['CDS-0Ab8A9', 'tumor_bam_alignment_summary_metrics']

In [ ]:
allstat_use['source'].value_counts()

In [ ]:
allstat.loc[:, ['internal_bam_filepath', 'source', 'sex', 'primary_disease', 'tumor_bam_alignment_summary_metrics']].to_csv("all.stats")

In [ ]:
clf = pd.read_table("23Q2.called.seg")
cle = pd.read_table("all.called.seg")
clf_seg_num = clf.groupby('Sample').apply(lambda x:x.shape[0])
cle_seg_num = cle.groupby('Sample').apply(lambda x:x.shape[0])

In [ ]:
cle_seg_num.index.isin(align_res.columns).sum()

In [ ]:
cle_seg_num_df = pd.DataFrame(cle_seg_num)

In [ ]:
cle_seg_num_df['SampleID'] = cle_seg_num_df.index

In [ ]:
align_res = align_res.T

In [ ]:
align_res['SampleID'] = align_res.index

In [ ]:
res = cle_seg_num_df.merge(align_res, left_on='SampleID', right_on='SampleID')

In [ ]:
cle_seg_num.plot.hist(bins=200, label='CCLE')
clf_seg_num.plot.hist(bins=200, label='CCLF')

In [ ]:
fig, ax = plt.subplots(1, 1)
sns.scatterplot(res, x='0_x', y=2, ax=ax)
ax.set_ylabel("Paired read Q>20")
ax.set_xlabel("Segment number")
ax.set_title("CCLE WES samples")
fig.set_size_inches(9, 5)

In [ ]:
newclf = pd.read_table("latest_segments/23Q2.called.seg")
newclf_seg_num = newclf.groupby('Sample').apply(lambda x:x.shape[0])

In [ ]:
newclf_seg_num

In [ ]:
cle_seg_num.plot.hist(bins=200, alpha=0.4, label='CCLE')
clf_seg_num.plot.hist(bins=200, alpha=0.4, label='CCLF')
newclf_seg_num.plot.hist(bins=200, alpha=0.4, label='New CCLF')

# Load meta json with alt-aware bams

## Load coverage analysis

In [ ]:
gcs_file_system = gcsfs.GCSFileSystem(project="broad-qqin")
gcs_json_path = "gs://broad-qqin-sparkles/BamMetrics/b012f426-0347-4433-8d16-f4298a134614/metadata.json"
with gcs_file_system.open(gcs_json_path) as f:
    json_dict = json.load(f)
depth2_output = [i['outputs']['result_sum'] for i in json_dict['calls']['BamMetrics.Depth2']]
depth2_output[:5]

In [ ]:
dep2_list = []
dep2_name = []
for dep2 in depth2_output:
#    print(dep2)
    dep2_name.append(os.path.basename(dep2).split('.')[0])
    with gcs_file_system.open(dep2) as f:
        dep2_list.append(float(f.readlines()[-1].strip().split()[-3].decode('utf-8')))

In [ ]:
plt.hist(dep2_list, bins=50)

In [ ]:
dep2_list = pd.DataFrame({"value": dep2_list, 'name': dep2_name})

In [ ]:
depth3_output = [i['outputs']['HsMetrics'] for i in json_dict['calls']['BamMetrics.CollectHsMetrics']]
depth3_output[:5]

In [ ]:
dep3_list = []
for dep3 in depth3_output:
#    print(dep2)
    dep3_list.append(pd.read_table(dep3, skiprows=6, nrows=1).loc[:, ["BAIT_SET", "MEAN_BAIT_COVERAGE"]])

In [ ]:
dep3_list = pd.concat(dep3_list, axis=0)

In [ ]:
dep2_list.head()

In [ ]:
dep3_list.head()

In [ ]:
dep3_list.merge(dep2_list, left_on='BAIT_SET', right_on='name').corr()

In [ ]:
sns.scatterplot(x='MEAN_BAIT_COVERAGE', y='value', data=dep3_list.merge(dep2_list, left_on='BAIT_SET', right_on='name'))
plt.ylabel("mosdepth on bait")

In [ ]:
plt.hist(dep3_list.iloc[:, 1], bins=50)

In [ ]:
depth_output = [i['stdout'] for i in json_dict['calls']['BamMetrics.Depth']]
cache_list = []
for dep1 in depth_output:
    print(dep1)
    try:
        with gcs_file_system.open(dep1) as f:
            dep1_list.append(float(f.read().strip().split()[-1].decode('utf-8')))    
    except:
        # cache files are in different folders
        with gcs_file_system.open(dep1.replace('stdout', 'cacheCopy/call_caching_placeholder.txt')) as f:
            cache_list.append(os.path.join(f.readlines()[-2].strip().split()[-1].decode('utf-8'), 'stdout'))
dep1_list = []
for dep1 in cache_list:
    with gcs_file_system.open(dep1) as f:
        dep1_list.append(float(f.read().strip().split()[-1].decode('utf-8')))  

In [ ]:
plt.hist(dep1_list, bins=50)

# old bams (not alt-aware with high segments)

In [ ]:
gcs_file_system = gcsfs.GCSFileSystem(project="broad-qqin")
gcs_json_path = "gs://broad-qqin-sparkles/BamMetrics/fbaa56e1-3ed9-4cc8-9c15-6d271df7f745/metadata.json"
with gcs_file_system.open(gcs_json_path) as f:
    json_dict = json.load(f)
depth_output = [i['stdout'] for i in json_dict['calls']['BamMetrics.Depth']]
dep1_list = []
for dep1 in depth_output:
    with gcs_file_system.open(dep1) as f:
        dep1_list.append(float(f.read().strip().split()[-1].decode('utf-8')))    

In [ ]:
plt.hist(dep1_list, bins=50)

## Old bam do not have Depth2 from mosdepth
#### depth2_output = [i['outputs']['result_sum'] for i in json_dict['calls']['BamMetrics.Depth2']]

# HsMetrics/samtools depth/mosdepth read in coverage comparison

In [ ]:
#test pandas API
pd.read_table('gs://broad-qqin-sparkles/BamMetrics/b1930991-8c01-4ca4-bdf8-55b8302b1c28/call-CollectHsMetrics/shard-*/*.hg38.hs_metrics.txt', skiprows=6, nrows=1)

In [ ]:
from google.cloud import storage
import os
client = storage.Client()
hsMetrics = []
for blob in client.list_blobs('broad-qqin-sparkles', prefix='BamMetrics'):
    if 'hs_metrics' in blob.name:
        hsMetrics.append(pd.read_table(os.path.join("gs://broad-qqin-sparkles/", blob.name), skiprows=6, nrows=1))
hsMetrics = pd.concat(hsMetrics, axis=0)

In [ ]:
hsMetrics.MEAN_TARGET_COVERAGE

In [ ]:
hsMetrics.MEDIAN_TARGET_COVERAGE

In [ ]:

# client = storage.Client()
# hsMetrics = []
# for blob in client.list_blobs('broad-qqin-sparkles', prefix='BamMetrics'):
#     if 'alignment_summary_metrics' in blob.name:
#         print(blob.name)
#         hsMetrics.append(pd.read_table(os.path.join("gs://broad-qqin-sparkles/", blob.name), nrows=1,header=None,sep=' ').iloc[[2], :])
# hsMetrics = pd.concat(hsMetrics, axis=0)

## Library complexity

In [ ]:
from google.cloud import storage
import os
client = storage.Client()
hsMetrics = []
for blob in client.list_blobs('broad-qqin-sparkles', prefix='BamMetrics/5a863ea2-c8a6-4d24-a56c-c8984c5a125e'):
    if 'libcomplex' in blob.name:
        print(blob.name)
        try:
            df = pd.read_table(os.path.join("gs://broad-qqin-sparkles/", blob.name), skiprows=6, nrows=1)
            print(df.head())
            hsMetrics.append(df)
        except:
            continue
hsMetrics = pd.concat(hsMetrics, axis=0)

In [ ]:
plt.hist(hsMetrics.PERCENT_DUPLICATION)

## Raw Read count comparison between alt and non-alt bams

In [ ]:
from google.cloud import storage
import os
client = storage.Client()
flagstatMetrics = []
names = []
for blob in client.list_blobs('broad-qqin-sparkles', prefix='BamMetrics/c3f68d21-844c-46d7-89cd-7ee6b4e719e2'):
    if 'flagstat' in blob.name:
        names.append(os.path.basename(blob.name).split('.')[0])
        flagstatMetrics.append(pd.read_table(os.path.join("gs://broad-qqin-sparkles/", blob.name), 
                                             header=None, nrows=1, delimiter=' ').iloc[:, :3])
flagstatMetrics = pd.concat(flagstatMetrics, axis=0)

In [ ]:
flagstatMetrics.index = names

In [ ]:
flagstatMetrics.head()

In [ ]:
flagstatMetrics.shape

In [ ]:

client = storage.Client()
flagstatMetrics_noalt = []
names = []
#broad-qqin-sparkles/BamMetrics/5a863ea2-c8a6-4d24-a56c-c8984c5a125e
for blob in client.list_blobs('broad-qqin-sparkles', prefix='BamMetrics/5a863ea2-c8a6-4d24-a56c-c8984c5a125e'):
    if 'flagstat' in blob.name:
        names.append(os.path.basename(blob.name).split('.')[0])
        flagstatMetrics_noalt.append(pd.read_table(os.path.join("gs://broad-qqin-sparkles/", blob.name), 
                                             header=None, nrows=1, delimiter=' ').iloc[:, :3])
flagstatMetrics_noalt = pd.concat(flagstatMetrics_noalt, axis=0)

In [ ]:
flagstatMetrics_noalt.index = names

In [ ]:
flagstatMetrics_noalt.head()

In [ ]:
flagstatMetrics_noalt.head()

In [ ]:
flagstatMetrics_noalt_final = flagstatMetrics_noalt.merge(flagstatMetrics, left_index=True, right_index=True)

In [ ]:
flagstatMetrics_noalt_final.head()

In [ ]:
sns.scatterplot(flagstatMetrics_noalt_final, x='0_x', y='0_y')
plt.ylabel("alt aware")
plt.xlabel("original")

# Alignment summary/samtools comparison for high quality reads

In [ ]:
# new bams
import re

gcs_file_system = gcsfs.GCSFileSystem(project="broad-qqin")

client = storage.Client()
flagstatMetrics = []
names = []
for blob in client.list_blobs('broad-qqin-sparkles', prefix='BamMetrics/c3f68d21-844c-46d7-89cd-7ee6b4e719e2'):    
    if 'stdout' in blob.name and 'call-Count' in blob.name:
        if 'pipelines-logs' not in blob.name:
            with gcs_file_system.open(os.path.join("gs://broad-qqin-sparkles/", blob.name).replace('stdout', 'Count-0.log')) as f:                
                target_line = f.readlines()[4].decode('utf-8')
                name = target_line.split('/')[-1]
                names.append(name)
#                print(os.path.join("gs://broad-qqin-sparkles/", blob.name).replace('stdout', 'Count-0.log'))
#                print(target_line)
#                print(name)
            with gcs_file_system.open(os.path.join("gs://broad-qqin-sparkles/", blob.name)) as f:
                flagstatMetrics.append(float(f.read().strip().split()[-1].decode('utf-8')))
#            break

In [ ]:
len(flagstatMetrics)

In [ ]:
len(names)

In [ ]:
plt.hist(flagstatMetrics, bins=20)

In [ ]:
# New bams
gcs_file_system = gcsfs.GCSFileSystem(project="broad-qqin")
gcs_json_path = "gs://broad-qqin-sparkles/BamMetrics/50f6e3b8-b1bb-4055-9508-42596cfa0d6d/metadata.json"
with gcs_file_system.open(gcs_json_path) as f:
    json_dict = json.load(f)
count_output = [i['outputs']['alignmentSummary'] for i in json_dict['calls']['BamMetrics.picardMetrics']]

In [ ]:
align_res_new = []
for align in count_output:
    align_res_new.append(pd.read_table(align, skiprows=6).loc[:, ['PF_HQ_ALIGNED_READS']])
align_res_new = pd.concat(align_res_new, axis=1)   
align_res_new.columns = list(map(lambda x:os.path.basename(x).split('.')[0], count_output))
count_output[:5]

In [ ]:
align_res_new = align_res_new.iloc[:3, :]

In [ ]:
align_res_new.shape

In [ ]:
##Old bams files 
gcs_file_system = gcsfs.GCSFileSystem(project="broad-qqin")
gcs_json_path = "gs://broad-qqin-sparkles/BamMetrics/01aad669-2562-4725-8170-c2d83452fcb0/metadata.json"
with gcs_file_system.open(gcs_json_path) as f:
    json_dict = json.load(f)
count_output = [i['outputs']['alignmentSummary'] for i in json_dict['calls']['BamMetrics.picardMetrics']]
align_res = []
for align in count_output:
    align_res.append(pd.read_table(align, skiprows=6).loc[:, ['PF_HQ_ALIGNED_READS']])
align_res = pd.concat(align_res, axis=1)   
align_res.columns = list(map(lambda x:os.path.basename(x).split('.')[0], count_output))
count_output[:5]    

In [ ]:
align_res = align_res.iloc[:3, :]
hq_res_df = pd.concat([align_res_new.T, align_res.T.loc[align_res_new.columns, :]], axis=1).iloc[:, [2, 5]]

In [ ]:
hq_res_df.columns = ['new', 'old']

In [ ]:
sns.scatterplot(x='new', y='old', data=hq_res_df)

In [ ]:
(hq_res_df.new - hq_res_df.old).hist()

In [ ]:
((hq_res_df.new - hq_res_df.old)/hq_res_df.new).hist()

In [ ]:
#new bams
samtools_count_df = pd.DataFrame({'names': list(map(lambda x:x.strip().split('.')[0], names)), 'flagstatMetrics': flagstatMetrics})

In [ ]:
samtools_count_df = samtools_count_df.loc[samtools_count_df.names!='monitoring', :]

In [ ]:
from scipy.stats import pearsonr
pearsonr(hq_res_df.loc[samtools_count_df.names, :].new, samtools_count_df.flagstatMetrics)

In [ ]:
pearsonr(hq_res_df.loc[samtools_count_df.names, :].old, samtools_count_df.flagstatMetrics)

In [ ]:
plt.scatter(hq_res_df.loc[samtools_count_df.names, :].new, samtools_count_df.flagstatMetrics)

In [ ]:
plt.scatter(hq_res_df.loc[samtools_count_df.names, :].old, samtools_count_df.flagstatMetrics)

# Coverage analysis for old and new bams

In [ ]:
def parse_output_from_wdlmeta(metafile: str="gs://broad-qqin-sparkles/BamMetrics/50f6e3b8-b1bb-4055-9508-42596cfa0d6d/metadata.json",
                              metric='depth') -> pd.DataFrame:
    """ new bam metafile gs://broad-qqin-sparkles/BamMetrics/50f6e3b8-b1bb-4055-9508-42596cfa0d6d/metadata.json
    old bam metafile gs://broad-qqin-sparkles/BamMetrics/01aad669-2562-4725-8170-c2d83452fcb0/metadata.json
    
    """
    gcs_file_system = gcsfs.GCSFileSystem(project="broad-qqin")
    with gcs_file_system.open(metafile) as f:
        json_dict = json.load(f)   

    if metric == 'depth':
        depth3_output = [i['outputs']['HsMetrics'] for i in json_dict['calls']['BamMetrics.CollectHsMetrics']]
        print(depth3_output[:5])
        dep3_list = []
        for dep3 in depth3_output:
            dep3_list.append(pd.read_table(dep3, skiprows=6, nrows=1).loc[:, ["BAIT_SET", "MEAN_BAIT_COVERAGE"]])
        df = pd.concat(dep3_list, axis=0)
    if metric == 'align':
        count_output = [i['outputs']['alignmentSummary'] for i in json_dict['calls']['BamMetrics.picardMetrics']]
        align_res = []
        for align in count_output:
            align_res.append(pd.read_table(align, skiprows=6).loc[:, ['PF_HQ_ALIGNED_READS']])
        df = pd.concat(align_res, axis=1)   
        df.columns = list(map(lambda x:os.path.basename(x).split('.')[0], count_output))        
    if metric == 'libcomplex':
        lib_output = [i['outputs']['metricsFile'] for i in json_dict['calls']['BamMetrics.Libcomplex']]
        lib_res = []
        names = []
        for lib in lib_output:
            try:
                lib_res.append(pd.read_table(lib, skiprows=6).loc[:, ['PERCENT_DUPLICATION']])     
                names.append(os.path.basename(lib).split('.')[0])
            except:
                print(lib)
                pass
        df = pd.concat(lib_res, axis=1)
        df.columns = names
        
    if metric == 'gc':
        gc_output = [i['outputs']['gcBiasSummary'] for i in json_dict['calls']['BamMetrics.picardMetrics']]
        gc_res = []
        names = []
        for gc in gc_output:
            try:
                gc_res.append(pd.read_table(gc, skiprows=6))
                names.append(os.path.basename(gc).split('.')[0])
            except:
                print(gc)
                pass
        print(len(gc_res))
        print(len(names))
        df = pd.concat(gc_res, axis=0)   
        df.index = names                 
    return df

In [ ]:
hs_dep_new = parse_output_from_wdlmeta()
hs_dep_old = parse_output_from_wdlmeta('gs://broad-qqin-sparkles/BamMetrics/01aad669-2562-4725-8170-c2d83452fcb0/metadata.json')

In [ ]:
fig, ax = plt.subplots(1, 4)
fig.set_size_inches(12, 3)
depths_df = hs_dep_old.merge(hs_dep_new, on='BAIT_SET')

sns.scatterplot(x='MEAN_BAIT_COVERAGE_x', y='MEAN_BAIT_COVERAGE_y',
                data=depths_df, ax=ax[0])
ax[0].set_xlabel('old bam')
ax[0].set_ylabel('new bam')
ax[0].plot([0, 1], [0, 1], transform=ax[0].transAxes)
ax[1].hist(depths_df.MEAN_BAIT_COVERAGE_x, bins=50, color='red')
ax[1].hist(depths_df.MEAN_BAIT_COVERAGE_y, bins=50, color='blue')

ax[3].hist(depths_df.MEAN_BAIT_COVERAGE_y-depths_df.MEAN_BAIT_COVERAGE_x, bins=50, color='blue')
depths_df.index = depths_df.BAIT_SET
depths_df.drop(columns=['BAIT_SET'], inplace=True)
depths = depths_df.melt()
depths['variable'] = depths['variable'] .map({"MEAN_BAIT_COVERAGE_x": 'old', 'MEAN_BAIT_COVERAGE_y': 'new'})
sns.boxplot(x='variable', y='value', data=depths, ax=ax[2])

In [ ]:
depths_df.head()

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)
sns.scatterplot(x=0, y='MEAN_BAIT_COVERAGE_y', 
                data=pd.concat([pd.DataFrame(newclf_seg_num), depths_df.loc[newclf_seg_num.index, :]], axis=1),
                ax=ax[0])
ax[0].set_title("New bams (alt aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Bait coverage")


sns.scatterplot(x=0, y='MEAN_BAIT_COVERAGE_x', 
                data=pd.concat([pd.DataFrame(clf_seg_num), depths_df.loc[clf_seg_num.index, :]], axis=1),
                ax=ax[1])
ax[1].set_title("Old bams (alt non-aware)")
ax[1].set_xlabel("Segment number")
ax[1].set_ylabel("Bait coverage")
ax[1].set_xlim(0, 3500)


In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)
sns.scatterplot(x=0, y='MEAN_BAIT_COVERAGE_y', 
                data=pd.concat([pd.DataFrame(newclf_seg_num), depths_df.loc[newclf_seg_num.index, :]], axis=1),
                ax=ax[0], label='new')
ax[0].set_title("Old/New bams (alt aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Bait coverage")


ax = sns.scatterplot(x=0, y='MEAN_BAIT_COVERAGE_x', 
                data=pd.concat([pd.DataFrame(clf_seg_num), depths_df.loc[clf_seg_num.index, :]], axis=1),
                ax=ax[0], label='old')


In [ ]:
align_new = parse_output_from_wdlmeta(metric='align')
align_old = parse_output_from_wdlmeta('gs://broad-qqin-sparkles/BamMetrics/01aad669-2562-4725-8170-c2d83452fcb0/metadata.json', metric='align')
align_new = align_new.iloc[:3, :]
align_old = align_old.iloc[:3, :]

In [ ]:
hq_res_df = pd.concat([align_old.T.loc[align_new.columns, :], align_new.T], axis=1).iloc[:, [2, 5]]
hq_res_df.columns = ['old', 'new']

fig, ax = plt.subplots(1, 4)
fig.set_size_inches(12, 3)

sns.scatterplot(x='old', y='new',
                data=hq_res_df, ax=ax[0])
ax[0].set_xlabel('old bam')
ax[0].set_ylabel('new bam')
ax[0].plot([0, 1], [0, 1], transform=ax[0].transAxes)

ax[1].hist(hq_res_df.new, bins=50, color='blue')
ax[1].hist(hq_res_df.old, bins=50, color='red')

ax[3].hist(hq_res_df.new-hq_res_df.old, bins=50, color='blue')

hq_res = hq_res_df.melt()
sns.boxplot(x='variable', y='value', data=hq_res, ax=ax[2])

In [ ]:
hq_res_df.head()

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)
sns.scatterplot(x=0, y='old', 
                data=pd.concat([pd.DataFrame(newclf_seg_num), hq_res_df.loc[newclf_seg_num.index, :]], axis=1),
                ax=ax[0])
ax[0].set_title("New bams (alt aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Bait coverage")


sns.scatterplot(x=0, y='new', 
                data=pd.concat([pd.DataFrame(clf_seg_num), hq_res_df.loc[clf_seg_num.index, :]], axis=1),
                ax=ax[1])
ax[1].set_title("Old bams (alt non-aware)")
ax[1].set_xlabel("Segment number")
ax[1].set_ylabel("Paired Read count (MAPQ>20)")
ax[1].set_xlim(0, 3500)


In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)
sns.scatterplot(x=0, y='new', 
                data=pd.concat([pd.DataFrame(newclf_seg_num), hq_res_df.loc[newclf_seg_num.index, :]], axis=1),
                ax=ax[0], alpha=0.4, label='new cclf')
ax[0].set_title("New bams (alt aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Bait coverage")


sns.scatterplot(x=0, y='old', 
                data=pd.concat([pd.DataFrame(clf_seg_num), hq_res_df.loc[clf_seg_num.index, :]], axis=1),
                ax=ax[0], alpha=0.4, label='old cclf')
ax[0].set_title("Old/New bams (alt non-aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Paired read Q>20")
ax[0].set_xlim(0, 3500)

sns.scatterplot(res, x='0_x', y=2, ax=ax[0], alpha=0.4, label='ccle')
# ax[1].set_ylabel("Paired read Q>20")
# ax[1].set_xlabel("Segment number")
# ax[1].set_title("CCLE WES samples")
# fig.set_size_inches(9, 5)

In [ ]:
lib_new = parse_output_from_wdlmeta(metric='libcomplex')
lib_old = parse_output_from_wdlmeta('gs://broad-qqin-sparkles/BamMetrics/01aad669-2562-4725-8170-c2d83452fcb0/metadata.json', metric='libcomplex')

In [ ]:
lib_df = pd.concat([lib_old.iloc[[0], :].T,lib_new.iloc[[0], :].T], axis=1)
lib_df.columns = ['old', 'new']
fig, ax = plt.subplots(1, 4)
fig.set_size_inches(12, 3)

sns.scatterplot(x='old', y='new',
                data=lib_df, ax=ax[0])
ax[0].set_xlabel('old bam')
ax[0].set_ylabel('new bam')
ax[0].plot([0, 1], [0, 1], transform=ax[0].transAxes)

ax[1].hist(lib_df.new, bins=50, color='blue')
ax[1].hist(lib_df.old, bins=50, color='red')

ax[3].hist(lib_df.new-lib_df.old, bins=50, color='blue')

lib = lib_df.melt()
sns.boxplot(x='variable', y='value', data=lib, ax=ax[2])

In [ ]:
lib_index = np.intersect1d(newclf_seg_num.index, lib_df.index)

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)
sns.scatterplot(x=0, y='new', 
                data=pd.concat([pd.DataFrame(newclf_seg_num).loc[lib_index, :], lib_df.loc[lib_index, :]], axis=1),
                ax=ax[0], alpha=0.4, label='new cclf')
ax[0].set_title("New bams (alt aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Bait coverage")


sns.scatterplot(x=0, y='old', 
                data=pd.concat([pd.DataFrame(clf_seg_num).loc[lib_index, :], lib_df.loc[lib_index, :]], axis=1),
                ax=ax[0], alpha=0.4, label='old cclf')
ax[0].set_title("Old/New bams (alt non-aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("Bait coverage")
ax[0].set_xlim(0, 3500)

# sns.scatterplot(res, x='0_x', y=2, ax=ax[0], alpha=0.4, label='ccle')
# ax[1].set_ylabel("Paired read Q>20")
# ax[1].set_xlabel("Segment number")
# ax[1].set_title("CCLE WES samples")
# fig.set_size_inches(9, 5)

In [ ]:
gc_new = parse_output_from_wdlmeta(metric='gc')

In [ ]:
gc_old = parse_output_from_wdlmeta('gs://broad-qqin-sparkles/BamMetrics/01aad669-2562-4725-8170-c2d83452fcb0/metadata.json', metric='gc')

In [ ]:
gc_new.head()

In [ ]:
gc_old.head()

In [ ]:
gc_new.shape, newclf_seg_num.shape

In [ ]:
pd.concat([pd.DataFrame(newclf_seg_num), gc_new.loc[newclf_seg_num.index, 'GC_DROPOUT']], axis=1)

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 6)
sns.scatterplot(x=0, y='GC_DROPOUT', 
                data=pd.concat([pd.DataFrame(newclf_seg_num), gc_new.loc[newclf_seg_num.index, 'GC_DROPOUT']], axis=1),
                ax=ax[0], alpha=0.4, label='new cclf')
ax[0].set_title("New bams (alt aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("GC_DROPOUT")


sns.scatterplot(x=0, y='GC_DROPOUT', 
                data=pd.concat([pd.DataFrame(clf_seg_num).loc[lib_index, :], gc_old.loc[lib_index, :]], axis=1),
                ax=ax[0], alpha=0.4, label='old cclf')
ax[0].set_title("Old/New bams (alt non-aware)")
ax[0].set_xlabel("Segment number")
ax[0].set_ylabel("GC_DROPOUT")
# ax[0].set_xlim(0, 3500)